In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandasql import sqldf
from sklearn import linear_model
import statsmodels.api as sm
from scipy import stats

In [6]:
class Assets:
  data = pd.read_sas('data.sas7bdat', encoding='latin-1')
  ff4data = pd.read_sas('ff4data.sas7bdat', encoding='latin-1')
  # train_start = '1975-01-01'
  # train_end = '2005-12-31'
  # test_start = '2006-01-01'
  # test_end = '2020-12-31'
  factors = []
  
  def __init__(self):
    self.data['PERMNO'] = self.data['PERMNO'].astype(int)
    self.data['year'] = self.data['DATE'].dt.year
    self.data['month'] = self.data['DATE'].dt.month
    self.data['monthid'] = (self.data['year']-1975)*12 + self.data['month']
    self.data['PRC'] = self.data['PRC'].apply(lambda x: x if x > 0 else x * -1)
    
    self.illiquidity_filter()
    
  def illiquidity_filter(self):
    self.data = self.data[self.data['PRC'] >= 5]
    self.data['MKTCAP'] = self.data['PRC'] * self.data['SHROUT']
    tmp = {}
    grp = self.data[(self.data['month'] == 1) & (self.data['MKTCAP'] >= 100000)].groupby('year')
    for yr, group in grp:
      tmp[yr] = list(group['PERMNO'])
    liquidity = self.data.groupby(['year'])['PERMNO'].transform(lambda x: x.isin(tmp[x.name]))
    self.data = self.data[liquidity]
  
  def standardize_factors(self):
    for factor in self.factors:
      continue

In [7]:
assets = Assets()

<ipython-input-6-33c6e1708a4b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['MKTCAP'] = self.data['PRC'] * self.data['SHROUT']


In [8]:
assets.data.sort_values('monthid')

,PERMNO,DATE,SHRCD,EXCHCD,PRC,RET,SHROUT,year,month,monthid,MKTCAP
256393,12626,1975-01-31,10.0,1.0,22.750000,0.309353,11404.0,1975,1,1,2.594410e+05
1015216,48274,1975-01-31,11.0,1.0,22.125000,0.264286,12724.0,1975,1,1,2.815185e+05
441362,17815,1975-01-31,11.0,3.0,31.625000,0.069536,8034.0,1975,1,1,2.540752e+05
314734,14189,1975-01-31,10.0,1.0,28.500000,0.266667,16176.0,1975,1,1,4.610160e+05
900727,42471,1975-01-31,10.0,3.0,12.250000,0.324324,13189.0,1975,1,1,1.615652e+05
...,...,...,...,...,...,...,...,...,...,...,...
412755,16693,2020-12-31,12.0,1.0,32.509998,0.139502,181406.0,2020,12,552,5.897509e+06
412991,16696,2020-12-31,11.0,1.0,11.300000,0.011638,67386.0,2020,12,552,7.614618e+05
413035,16697,2020-12-31,11.0,1.0,10.670000,0.302808,86845.0,2020,12,552,9.266362e+05
413152,16703,2020-12-31,11.0,3.0,23.000000,0.012675,11198.0,2020,12,552,2.575540e+05
